# Coincaster - Get mongodb running on a Raspberry Pi 3

To store the data for the coincaster project, we will use a raspberry pi 3. In this notebook, all the steps to get the mongo database running on the raspberry pi 3 (RPi3) are covered. We will also allow remote access to the database and connect to it using pymongo.



# Table of contents

### [1. Set up Ubuntu Server on Raspberry Pi 3](#1)
### [2. Install and set up mongodb](#2)
### [3. Final steps on the Raspberry Pi](#3)

<a id='1'></a>
## 1. Set up Ubuntu Server on Raspberry Pi 3
As operating system, we will use Ubuntu Server, because for the NoSQL database to be used, Rasbian only supports version 2.4, which is inferior to the mongodb 3.x version that is available on Ubuntu Server.


You can download it here and follow the instructions to get the image on the microSD card (depends on your operating system):

https://ubuntu.com/download/raspberry-pi

Once Ubuntu server is installed on the RPi3, we connect to it using the username `ubuntu` and password `ubuntu`. It needs to be changed on the first login. After updating (`apt install update` and `apt install upgrade`), remote access via `ssh` should be enabled, this is achieved with the commands:

```
sudo apt install openssh-server
sudo systemctl status ssh
sudo systemctl start ssh
```

The server can then be accessed by:

```
ssh username@ip_address
```

The IP address is found using the command `ip a` when logged in to the RPi3, in case it is not known.

It is also recommended to use an ssh-key to log in which makes logging in easier. Information can be found here: https://www.digitalocean.com/community/tutorials/how-to-set-up-ssh-keys-on-ubuntu-1604.


<a id='2'></a>
## 2. Install and set up mongodb

### Install mongodb
To install `mongodb`, follow the steps laid out in the official mongoDB documentation:

https://docs.mongodb.com/manual/tutorial/install-mongodb-on-debian/

It may be that you need to use the commands for Debian 9 "Stretch".

### Set up mongodb

After mongodb is installed, we have to set it up for remote access. To do that, change the configuration file (requires elevated rights):

```
sudo nano /etc/mongod.conf
```

There, you need to change the following:

- `bind_ip = 0.0.0.0` to allow remote, non local connections
- `auth = true` to turn on security (requires password to log in)

Next, we want to add a user to be able to log in to the mongo db remotely:
We chose to add this user as admin
```
use admin;
```
```
db.createUser({
      user: "USERNAME",
      pwd: "PASSWORD",
      roles: [
                { role: "userAdminAnyDatabase", db: "admin" },
                { role: "readWriteAnyDatabase", db: "admin" },
                { role: "dbAdminAnyDatabase",   db: "admin" }
             ]
  });
```

Now that the user is admin, we have to add it to the database. For this project, the database will be named `coincaster`.

```
use coincaster;
```
```
db.createUser({
      user: "USERNAME",
      pwd: "PASSWORD",
      roles: [
                { role: "userAdmin", db: "coincaster" },
                { role: "dbAdmin",   db: "coincaster" },
                { role: "readWrite", db: "coincaster" }
             ]
  });
```

That's it, remote access should now be possible. To test, you can simply connect by using `pymongo`. The script below will connect to the database and list the collections. Note that you need to provide the correct credentials.

In [12]:
import pymongo
import pandas as pd

cred = '../01.Original_data/credentials/mongodb.pw'
data_cred = pd.read_csv(cred, index_col=0, header=None)
USER = data_cred.loc['USER'].values[0]
DB = data_cred.loc['DB'].values[0]
PW = data_cred.loc['PW'].values[0]
HOST = data_cred.loc['HOST'].values[0]

client = pymongo.MongoClient(f"mongodb://{USER}:{PW}@{HOST}/{DB}") # defaults to port 27017

db = client.coincaster

# list first 5 collections
db.list_collection_names()[:5]

['ThoreNext__THX.csv',
 'Metadium__META.csv',
 'SynchroBitcoin__SNB.csv',
 'Telcoin__TEL.csv',
 'Verasity__VRA.csv']

<a id='3'></a>
## 3. Final steps on the Raspberry Pi

Lastly, we will also install some python packages on the RPi3 that we will use later for the data mining. Note that especially pandas will take a long time and it is recommended to install it while being directly connected to the RPi3 and not via SSH. For me, the system froze due to memory limits (installation still completed after about 2-3 hours).

```
sudo apt-get install python3-wheel
sudo apt-get install python3-numpy
sudo apt-get install python3-pandas
python3 -m pip install beautifulsoup4
python3 -m pip install requests
python3 -m pip install pymongo
```



That's it, the system is now good to go!